# Vitamin Example

![](http://slideplayer.com/3006727/11/images/56/Vitamin+C+for+colds+Vitamin+C+is+necessary+for+making+collagen%2C+and+for+many+body+functions..jpg)

https://newonlinecourses.science.psu.edu/stat504/node/74/

https://newonlinecourses.science.psu.edu/stat504/sites/onlinecourses.science.psu.edu.stat504/files/lesson03/VitaminC/index.R

In [1]:
import pandas as pd
import numpy as np

# import plotting libraries
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
plt.style.use('seaborn-white')

# Statistical Packages
import scipy.stats as stats
import statsmodels.api as sm
import statsmodels.formula.api as smf


In [2]:
vitamin = pd.DataFrame(columns=['Cold', "No Cold"], index=['Placebo', 'Vitamins'])
vitamin.loc['Placebo'] = [31, 109]
vitamin.loc['Vitamins'] = [17, 122]
table = sm.stats.Table(vitamin)
print(table.table_orig)

          Cold  No Cold
Placebo     31      109
Vitamins    17      122


### Chi-square Test of Independence
https://www.statsmodels.org/stable/contingency_tables.html

Independence is the property that the row and column factors occur independently. Association is the lack of independence. If the joint distribution is independent, it can be written as the outer product of the row and column marginal distributions:

$P_{ij} = \sum_k P_{ij} \cdot \sum_k P_{kj} \quad \text{for all} \quad  i, j$

This function computes the chi-square statistic and p-value for the hypothesis test of independence of the observed frequencies in the contingency table observed. 



#### Pearson Residuals

__How large is the discrepancy between the two proposed models.__ The residuals provide an indication on where the diffreneces are and potentially which leads to a rejection in the null hyptoehsioes. 

$r_j=\dfrac{X_j-n\hat{\pi}_j}{\sqrt{n\hat{\pi}_j}}=\dfrac{O_j-E_j}{\sqrt{E_j}}$


The __Pearson residual__ compares the observed with the expected counts. The sign (+/-) indicates whether the observed requency in cell _j_ is higher or lower than the value fitted under the model, and the magnitude indicates the degree of departure. 

In [3]:
print("\nExpected: ")
print(table.fittedvalues)

print("\nResiduals: ")
print(table.resid_pearson)

print("\nmarginal_probabilities: ")
print(table.marginal_probabilities)


Expected: 
               Cold     No Cold
Placebo   24.086022  115.913978
Vitamins  23.913978  115.086022

Residuals: 
              Cold   No Cold
Placebo   1.408787 -0.642185
Vitamins -1.413846  0.644491

marginal_probabilities: 
(Placebo     0.501792
Vitamins    0.498208
dtype: float64, Cold       0.172043
No Cold    0.827957
dtype: float64)


When reviewing the absolute value, |$r_{j}$| is much larger than $\sqrt{(k-1)/k}$ ~ 2 or more, then the model __does not appear to fit well for cell__ _j_. Above we can reveiw the pearson residuals to __identify those cells that do not fit the expected model as closely as we might have assumed__. 

[STAT504 notes](https://newonlinecourses.science.psu.edu/stat504/node/62/)

#### Peasron Chi-squared test WITHOUT Yates' continuity correction

Above are the tables for the expected values and the residuals.

In [4]:
rslt = table.test_nominal_association()
print("Chi-Square Statistic: ", rslt.statistic)
print("Chi-Square Probability (p-value): ", rslt.pvalue)

Chi-Square Statistic:  4.811412646320789
Chi-Square Probability (p-value):  0.0282718602468226


__The test for independence yields χ2 = 4.814 and the p-value=0.0283. The conclusion here would be that there is strong evidence against the null hypothesis of independence. The statistic χ2 = 4.814 means that the evidence against the null is strong.__ 
 
Yates' continuity correction. This correction subtracts 0.5 from a difference between the observed and the expected counts in the formula for the χ2 statistic, e.g., {oij-nij}-0.5. It is used in situations when there are cells with small expected counts (e.g., less than 5) in order to better approximate exact inference tests. 

__Fisher Exact Test__

In [5]:
oddsratio, pvalue = stats.fisher_exact(vitamin, alternative='two-sided')
print("Odds Ratio: ", oddsratio)
print("p-value   : ", pvalue)

oddsratio, pvalue = stats.fisher_exact(vitamin, alternative='greater')
print("\nLeft Tail prob: ",pvalue)
oddsratio, pvalue = stats.fisher_exact(vitamin, alternative='less')
print("\nRight Tail prob: ",pvalue)

Odds Ratio:  2.0410145709660013
p-value   :  0.03849249137522522

Left Tail prob:  0.020522715992754487

Right Tail prob:  0.9910066932152484


* __Left tail probability__ is the probability of all tables such that the (1,1) cell value is less than or equal to the one observed; _i.e., left tail p-value_
* __Right rail p-value__ probability of all tables iwht (1,1) cell value grater than or equal to the observed. 

In [6]:
table = np.array(vitamin)
t22 = sm.stats.Table2x2(table)
print(t22.table_orig)
print(t22.summary())

[[ 31 109]
 [ 17 122]]
               Estimate   SE   LCB   UCB  p-value
-------------------------------------------------
Odds ratio        2.041       1.070 3.892   0.030
Log odds ratio    0.713 0.329 0.068 1.359   0.030
Risk ratio        1.811       1.052 3.116   0.032
Log risk ratio    0.594 0.277 0.051 1.137   0.032
-------------------------------------------------


### Interpretation 

Therefore, odds ratio = 2.041.... $\hat{\theta}=0.49$ _(1/.49)_ means that:
* the odds of getting a cold given `vitamin C` are __.49 time the odds of getting cold given the placebo__
* the odds of getting a cold given a placebo are 1/.49 = __2.04 time greater than the odds given vitamin C__
* getting cold is less likely given vitamin C than given a placebo

Row Percentage

In [7]:
table / table.sum(axis=1)

# table / table.sum(axis=0) # column percentages
# table / table.sum() # overall percentage

array([[0.22142857, 0.78417266],
       [0.12142857, 0.87769784]])

With the row percentages, we can conclude:
* odds of "success" _(i.e., getting a cold)_ given that the skier took vitamin C, 0.12 / 0.88 = 0.14
* odds of "success" _(i.e., getting a cold)_ given that a skier took a placebo pill, 0.22 / 0.78 = 0.28

The odds ratio is 0.14/0.28 = 0.48


### Point Estimates, CI and HT

__Testing H0 : θ = 1 is equivalent to testing independence in 2 × 2 tables.__

Odds of getting a cold vs not getting a cold given that a person took a `placebo`

$odds_1=\dfrac{P(Z=1|Y=1)}{P(Z=2|Y=1)}=\dfrac{\pi_{1|1}}{\pi_{2|1}}=\dfrac{\pi_{1|1}}{1-\pi_{1|1}}$

__Column 1 Risk Estmates__

In [8]:
ColSum_1 = table[:2,0].sum()
ColSum_2 = table[:2,1].sum()
row_sum = table.sum()
row_sum1 =table[:1].sum()
row_sum2 = table[1:].sum()
risk1_col1 = table[0,0] / row_sum1
risk2_col1 = table[1,0] / row_sum2
rho1 = risk1_col1 / risk2_col1
total1 = ColSum_1 / row_sum
diff1 = risk2_col1 - risk1_col1

print("Placebo:")
print("Risk estimate row 1: ", risk1_col1)
print("Risk estimate row 2: ", risk2_col1)
print("Col 1 Risk Total: ", total1)
print("Difference row 1-row2: ", diff1)

Placebo:
Risk estimate row 1:  0.22142857142857142
Risk estimate row 2:  0.1223021582733813
Col 1 Risk Total:  0.17204301075268819
Difference row 1-row2:  -0.09912641315519012


__Test the odds of "success"__ in other words _getting a cold_ given that the skier took vitamin C:

__The confidence interval for the difference in proportions for column 1__


$\text{log}\hat{\theta} \pm 1.96\sqrt{\dfrac{1}{n_{11}}+\dfrac{1}{n_{12}}+\dfrac{1}{n_{21}}+\dfrac{1}{n_{22}}}$

In [9]:
CI_significance = stats.norm.ppf(.975)
se_diff1 = np.sqrt(risk1_col1*(1-risk1_col1)/row_sum1+ risk2_col1*(1-risk2_col1)/row_sum2)
ci_diff1_low = diff1 - CI_significance * se_diff1
ci_diff1_hi = diff1 + CI_significance * se_diff1

print("Placebo SE diff: ", se_diff1)
print("CI diff :", ci_diff1_low, ci_diff1_hi )


Placebo SE diff:  0.04476243330709762
CI diff : -0.18685917029747762 -0.01139365601290264


__Column 2 Risk Estmates__

The second odds given that Vitamin C was taken:

$odds_2=\dfrac{P(Z=1|Y=2)}{P(Z=2|Y=2)}=\dfrac{\pi_{1|2}}{\pi_{2|2}}=\dfrac{\pi_{1|2}}{1-\pi_{1|2}}$

In [10]:
ColSum_2 = table[:2,1].sum()
row_sum = table.sum()
risk1_col2 = table[0,1] / row_sum1
risk2_col2 = table[1,1] / row_sum2
rho2 = risk1_col2 / risk2_col2
total2 = ColSum_2 / row_sum
diff2 = risk2_col2 - risk1_col2

print("Placebo:")
print("Risk estimate row 1: ", risk1_col2)
print("Risk estimate row 2: ", risk2_col2)
print("Col 1 Risk Total: ", total2)
print("Difference row 1-row2: ", diff2)

Placebo:
Risk estimate row 1:  0.7785714285714286
Risk estimate row 2:  0.8776978417266187
Col 1 Risk Total:  0.8279569892473119
Difference row 1-row2:  0.09912641315519011


__The confidence interval for the difference in proportions for column 2__

In [11]:
CI_significance = stats.norm.ppf(.975)
se_diff2 = np.sqrt(risk1_col2*(1-risk1_col2)/row_sum1+ risk2_col2*(1-risk2_col2)/row_sum2)
ci_diff2_low = diff2 - CI_significance * se_diff2
ci_diff2_hi = diff2 + CI_significance * se_diff2

print("Placebo SE diff: ", se_diff2)
print("CI diff :", ci_diff2_low, ci_diff2_hi )

Placebo SE diff:  0.04476243330709762
CI diff : 0.011393656012902625 0.1868591702974776


__Estimate of the Odds of the two rows__

In [12]:
odds1 = (table[1,0] / row_sum2) / (table[0,0] / row_sum1)
odds2 = (table[1,1] / row_sum2) / (table[0,1] / row_sum1)

odds_ratio = odds1 / odds2
print("Odds Ratio: ", odds_ratio)
print("Odds1: ", odds1)
print("Odds2: ", odds2)

Odds Ratio:  0.4899524061343205
Odds1:  0.5523323276862382
Odds2:  1.12731832882318


__Odds ratio:__

$\text{log}(0.490)\pm 1.96 \sqrt{1/17+1/109+1/122+1/31}$

In [13]:
CI_odds_ratio_low = np.exp(np.log(odds_ratio) - CI_significance * np.sqrt((1/table).sum()))
CI_odds_ratio_hi = np.exp(np.log(odds_ratio) + CI_significance * np.sqrt((1/table).sum()))



print("Odds Ratio: ", odds_ratio)
print("CI low", CI_odds_ratio_low)
print("CI high", CI_odds_ratio_hi)

Odds Ratio:  0.4899524061343205
CI low 0.2569419145023999
CI high 0.9342709255580329


### Interpretation 

Therefore, $\hat{\theta}=0.49$ means that:
* the odds of getting a cold given `vitamin C` are __.49 time the odds of getting cold given the placebo__
* the odds of getting a cold given a placebo are 1/.49 = __2.04 time greater than the odds given vitamin C__
* getting cold is less likely given vitamin C than given a placebo